In [ ]:
import pandas as pd
import gzip
import os
import numpy as np

In [ ]:
####################################################
############# Setear segun cada maquina ############
#os.chdir("C:/Users/herna/labo3_empresa3_repo/")
#os.chdir("C:/diego_tools/labo3/")
os.chdir("/home/dcastro_contacto/buckets/b1/")
####################################################

In [ ]:
arch_sellout = "datasets/tb_sellout_02.txt.gz"
arch_salida_prefijo = "datasets/productos_a_predecir"

In [ ]:
# Variables para definir que atributos se descartan
meses_para_control_vigencia = [201904,201905,201906] #meses en los cuales deben aparecer los productos para ser considerados vigentes (NO discontinuados) y ser tomados en la prediccion
tope_fecha_historia = 201902 #los productos que aparezcan desde este mes (inclusive) en adelante, se excluyen por tener poca historia de ventas

## Sellout

In [ ]:
# Abrir el archivo .gz y cargarlo en un DataFrame
with gzip.open(arch_sellout, 'rt') as archivo:
    # Leer el archivo línea por línea
    df_sellout = pd.read_csv(archivo,sep="\t")

In [ ]:
df_sellout.info()

In [ ]:
# Por las dudas, eliminamos duplicados
print(len(df_sellout))
df_sellout.drop_duplicates(inplace=True)
print(len(df_sellout))

**--> sin duplicados**

In [ ]:
df_sellout.isna().sum()

**--> sin nulos**

In [ ]:
df_sellout.periodo.unique()                

In [ ]:
len(df_sellout.product_id.unique())          

--> algunos productos no van a tener descripción

In [ ]:
len(df_sellout.customer_id.unique())            

In [ ]:
df_sellout.head()

## Descarte de Productos que no hay que predecir

In [ ]:
# Discontinuados (sin ventas en 3 meses mas adelante)
product_ids_vigentes =  df_sellout[df_sellout.periodo.isin(meses_para_control_vigencia)].product_id.unique()
print("Vigentes:", len(product_ids_vigentes))

product_ids_discontinuados = set(df_sellout.product_id.unique()).difference(set(product_ids_vigentes))
print("Discontinuados:", len(product_ids_discontinuados))

#Sin historia suficiente (minima fecha >= 201902)
product_ids_sin_hist_sufic = df_sellout.groupby("product_id").agg({"periodo":"min"}).reset_index()
product_ids_sin_hist_sufic = product_ids_sin_hist_sufic[product_ids_sin_hist_sufic.periodo>=tope_fecha_historia]
print(product_ids_sin_hist_sufic.head(5))
product_ids_sin_hist_sufic = product_ids_sin_hist_sufic.product_id.unique()
print("Menos 3 meses hist:", len(product_ids_sin_hist_sufic))

#Interseccion y union
print("Interseccion: ", len((set(product_ids_discontinuados).intersection(set(product_ids_sin_hist_sufic)))))
print("Union: ", len((set(product_ids_discontinuados).union(set(product_ids_sin_hist_sufic)))))


product_ids_para_predecir = set(df_sellout.product_id.unique()).difference((set(product_ids_discontinuados).union(set(product_ids_sin_hist_sufic))))
print("\nTotal:", len(product_ids_para_predecir))

In [ ]:
#Se guardan en un csv aparte
df_prods_prediccion = pd.DataFrame(data={"product_id":list(product_ids_para_predecir)})
df_prods_prediccion.to_csv(arch_salida_prefijo + "_" + str(tope_fecha_historia) + ".csv", index=False)